In [ ]:
#!pip install keras

# Create flowers dataset

In [ ]:
import os
import random
import shutil



# To create the same dataset
random.seed(0)

# Here write the path to your dataset

base = 'C:/Users/David_000/Documents/EDAN95'
original_dataset_dir = os.path.join(base, r'flowers')
dataset = os.path.join(base, r'flowers_split')

train_dir = os.path.join(dataset, r'train')
validation_dir = os.path.join(dataset, r'validation')
test_dir = os.path.join(dataset, r'test')

categories = os.listdir(original_dataset_dir)
categories = [category for category in categories if not category.startswith('.')]
print('Image types:', categories)
data_folders = [os.path.join(original_dataset_dir, category) for category in categories]

pairs = []
for folder, category in zip(data_folders, categories):
    images = os.listdir(folder)
    images = [image for image in images if not image.startswith('.')]
    pairs.extend([(image, category) for image in images])

random.shuffle(pairs)
img_nbr = len(pairs)
train_images = pairs[0:int(0.6 * img_nbr)]
val_images = pairs[int(0.6 * img_nbr):int(0.8 * img_nbr)]
test_images = pairs[int(0.8 * img_nbr):]

# print(train_images)
print(len(train_images))
print(len(val_images))
print(len(test_images))

for image, label in train_images:
    src = os.path.join(original_dataset_dir, label, image)
    dst = os.path.join(train_dir, label, image)
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    shutil.copyfile(src, dst)

for image, label in val_images:
    src = os.path.join(original_dataset_dir, label, image)
    dst = os.path.join(validation_dir, label, image)
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    shutil.copyfile(src, dst)

for image, label in test_images:
    src = os.path.join(original_dataset_dir, label, image)
    dst = os.path.join(test_dir, label, image)
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    shutil.copyfile(src, dst)

# Implement a Simple CNN

In [32]:
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import models
from keras import optimizers

import os
import random
import shutil
import numpy as np


In [28]:
base = 'C:/Users/David_000/Documents/EDAN95'
original_dataset_dir = os.path.join(base, r'flowers')
dataset = os.path.join(base, r'flowers_split')

train_dir = os.path.join(dataset, r'train')
validation_dir = os.path.join(dataset, r'validation')
test_dir = os.path.join(dataset, r'test')

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255)
val_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(150, 150),
    batch_size=25, class_mode= 'categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_dir, target_size=(150, 150),
    batch_size=5, class_mode= 'categorical')

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
input_shape=(150, 150, 3)))

model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))

model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))

model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())

model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(5, activation='sigmoid'))

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# model.compile(loss='binary_crossentropy',
#             optimizer=optimizers.RMSprop(lr=1e-4),
#             metrics=['acc'])

In [ ]:
history = model.fit_generator(train_generator, steps_per_epoch=105, epochs=10,
                            validation_data=validation_generator, validation_steps=173)

In [ ]:
model.save('simpleCNN1.h5')

### Create second model with dropout and softmax

In [ ]:
model2 = models.Sequential()
model2.add(layers.Conv2D(32, (3, 3), activation='relu',
input_shape=(150, 150, 3)))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Dropout(0.25))

model2.add(layers.Conv2D(128, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Dropout(0.25))

model2.add(layers.Conv2D(128, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Flatten())
model2.add(layers.Dense(512, activation='relu'))
model2.add(layers.Dense(5, activation='softmax'))

In [ ]:
model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history2 = model2.fit_generator(train_generator, steps_per_epoch=105, epochs=10,
                            validation_data=validation_generator, validation_steps=173)

### Save model

In [ ]:
model2.save('simpleCNN2.h5')

# Plot Accuracy

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

# Setting up data augmentation

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, 
                             shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
from keras.preprocessing import image

In [ ]:
# fnames = [os.path.join(train_dir, fname) for fname in os.listdir(train_dir)]

# img_path = fnames[3]
# img = image.load_img(img_path, target_size=(150, 150))
# x = image.img_to_array(img)
# x = x.reshape((1,) + x.shape)
# i=0
# for batch in datagen.flow(x, batch_size=1):
#     plt.figure(i)
#     imgplot = plt.imshow(image.array_to_img(batch[0]))
#     i += 1
#     if i % 4 == 0:
#         break
# plt.show()

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150, 150), 
                                                    batch_size=25, class_mode='categorical')


validation_generator = test_datagen.flow_from_directory(validation_dir,target_size=(150, 150),
                                                        batch_size=20,class_mode='categorical')

In [ ]:
model3 = models.Sequential()
model3.add(layers.Conv2D(32, (3, 3), activation='relu',
input_shape=(150, 150, 3)))
model3.add(layers.MaxPooling2D((2, 2)))

model3.add(layers.Conv2D(64, (3, 3), activation='relu'))
model3.add(layers.MaxPooling2D((2, 2)))
model3.add(layers.Dropout(0.25))

model3.add(layers.Conv2D(128, (3, 3), activation='relu'))
model3.add(layers.MaxPooling2D((2, 2)))
model3.add(layers.Dropout(0.25))

model3.add(layers.Conv2D(128, (3, 3), activation='relu'))
model3.add(layers.MaxPooling2D((2, 2)))

model3.add(layers.Flatten())
model3.add(layers.Dense(512, activation='relu'))
model3.add(layers.Dense(5, activation='softmax'))

In [ ]:
history3 = model3.fit_generator(train_generator, steps_per_epoch=105, epochs=10,
                            validation_data=validation_generator, validation_steps=20)

# Import NetWork from Keras

In [ ]:
from keras.applications.inception_v3 import InceptionV3

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = layers.Dense(512, activation = 'relu')(x)
preds = layers.Dense(5, activation = 'softmax')(x)

model_V3 = models.Model(inputs = base_model.input, outputs = preds)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255)
val_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(150, 150),
    batch_size=25, class_mode= 'categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_dir, target_size=(150, 150),
    batch_size=20, class_mode= 'categorical')

In [ ]:
model_V3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history_V3 = model_V3.fit_generator(train_generator, steps_per_epoch=105, epochs=10,
                            validation_data=validation_generator, validation_steps=20)

## Inception V3 with augment data

In [29]:
train_datagen_aug = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, 
                             shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen_aug.flow_from_directory(train_dir, target_size=(150, 150), 
                                                    batch_size=25, class_mode='categorical')


validation_generator = test_datagen.flow_from_directory(validation_dir,target_size=(150, 150),
                                                        batch_size=20,class_mode='categorical')

Found 2625 images belonging to 5 classes.
Found 865 images belonging to 5 classes.


In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = layers.Dense(512, activation = 'relu')(x)
preds = layers.Dense(5, activation = 'softmax')(x)

for layer in base_model.layers:
    layer.trainable = False

model_V3_aug = models.Model(inputs = base_model.input, outputs = preds)

model_V3_aug.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [60]:
base_model = InceptionV3(weights='imagenet', include_top=False)

final_model = models.Sequential()
final_model.add(layers.Dense(512, activation = 'relu', input_dim = 3*3*2048))
final_model.add(layers.Dropout(0.5))
final_model.add(layers.Dense(5, activation = 'softmax'))

final_model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [48]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 25

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 3, 3, 2048))
    labels = np.zeros(shape=(sample_count,5))
    generator = datagen.flow_from_directory(directory, target_size=(150, 150), batch_size=batch_size,class_mode='categorical')
    i=0
    for inputs_batch, labels_batch in generator:
        features_batch = base_model.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 850)
test_features, test_labels = extract_features(test_dir, 850)


Found 2625 images belonging to 5 classes.
Found 865 images belonging to 5 classes.
Found 865 images belonging to 5 classes.


In [54]:
train_features_f = np.reshape(train_features, (2000, 3*3* 2048))
validation_features_f = np.reshape(validation_features, (850, 3*3* 2048))
test_features_f = np.reshape(test_features, (850, 3*3* 2048))


# scores = model.evaluate()


In [61]:
history = final_model.fit(train_features_f, train_labels, epochs=10, 
                    batch_size=25, validation_data=(validation_features_f, validation_labels))


Train on 2000 samples, validate on 850 samples
Epoch 1/10
2000/2000 [==============================] - 10s 5ms/step - loss: 12.2943 - accuracy: 0.5625 - val_loss: 1.0422 - val_accuracy: 0.7012
Epoch 2/10
2000/2000 [==============================] - 7s 4ms/step - loss: 1.4281 - accuracy: 0.6790 - val_loss: 0.7066 - val_accuracy: 0.7541
Epoch 3/10
2000/2000 [==============================] - 7s 4ms/step - loss: 1.2076 - accuracy: 0.7305 - val_loss: 1.9908 - val_accuracy: 0.6118
Epoch 4/10
2000/2000 [==============================] - 7s 4ms/step - loss: 1.1501 - accuracy: 0.7405 - val_loss: 1.3955 - val_accuracy: 0.7235
Epoch 5/10
2000/2000 [==============================] - 7s 4ms/step - loss: 0.9444 - accuracy: 0.7975 - val_loss: 1.0722 - val_accuracy: 0.7506
Epoch 6/10
2000/2000 [==============================] - 7s 4ms/step - loss: 0.8068 - accuracy: 0.8160 - val_loss: 1.2505 - val_accuracy: 0.7624
Epoch 7/10
2000/2000 [==============================] - 7s 4ms/step - loss: 0.7846 - ac